Задание 1.

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [139]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime as dt

In [63]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'
req = requests.get(URL)
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [ ]:
Так как вариант с preview-информацией разбирался на занятии, сразу делал необязательное

In [58]:
all_links = []
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    title_element = post.find('a', class_='post__title_link')
    link = title_element.attrs.get('href') # получаем ссылку для каждого поста
    soup_temp = BeautifulSoup(requests.get(link).text, 'html.parser')
    text_link = soup_temp.find('div', class_='post__text').text #получаем содержимое поста по ссылке
    for search_word in KEYWORDS: 
        if search_word in text_link: #проверяем наличие слова в тексте поста
            all_links.append(link)
            break
print(all_links)

['https://habr.com/ru/post/517056/', 'https://habr.com/ru/post/517044/']


In [60]:
habr_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
    title = soup.find('span', class_='post__title-text').text
    text = soup.find('div', class_='post__text').text
    row = {'date': date, 'title': title, 'link': link, 'text': text}
    habr_news = pd.concat([habr_news, pd.DataFrame([row])]) 
habr_news   

,date,title,link,text
0,2020-08-29,Интерактивная визуализация алгоритмов на базе ...,https://habr.com/ru/post/517056/,Jupyter уже давно зарекомендовал себя как удоб...
0,2020-08-29,"Бот в телеграм, озвучивающий ваши эмоции в соо...",https://habr.com/ru/post/517044/,Привет!\nВ этой статье я опишу своего бота в т...


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [188]:
email_inf = pd.DataFrame()
URL = 'https://digibody.avast.com/v1/web/leaks'
EMAIL = ['xuk@mail.ru']
for emails in EMAIL:    
    res = requests.post(URL, json={'email': emails})
    for value in res.json()['value']:
        date_time = datetime.datetime.fromtimestamp(value['leak_info']['date']/1000)
        row = {'emails': value['username'], 'date': date_time, 'source': value['domain'], 'description': value['leak_info']['description']}
        email_inf = pd.concat([email_inf, pd.DataFrame([row])]) 
email_inf

,emails,date,source,description
0,xuk@mail.ru,2017-12-22 02:00:00,,The proliferation of stolen or leaked database...
0,xuk@mail.ru,2018-08-24 03:00:00,,This combolist was compiled from a variety of ...
0,xuk@mail.ru,2019-02-06 02:00:00,,"On January 7, 2019, an online user named Sanix..."
0,xuk@mail.ru,2017-08-30 03:00:00,sprashivai.ru,"In January 2015, Sprashivai.ru's user database..."
0,xuk@mail.ru,2019-02-06 02:00:00,,"On January 7, 2019, an online user named Sanix..."
0,xuk@mail.ru,2019-01-29 02:00:00,,"On January 7, 2019, an online user named Sanix..."
0,xuka,2019-08-08 03:00:00,borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
0,xuk@mail.ru,2019-02-06 02:00:00,,"On January 7, 2019, an online user named Sanix..."
0,xuk@mail.ru,2016-10-29 03:00:00,vk.com,Popular Russian social networking platform VKo...
0,xuk@mail.ru,2017-04-10 03:00:00,sprashivai.ru,"In May 2015, Ask RU's user database was leaked..."
